# This Notebook explores the Titanic dataset and tries to predict if a passenger died.

### Loading and Viewing the datasets. 

In [3]:
# Load libraries for EDA.
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [4]:
# Load required data
path = "../../../Data/titanic/"
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")

Let's peek at the three datasets and observe their features.

In [6]:
train.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

The attributes have the following meaning:

- PassengerId: a unique identifier for each passenger
- Survived: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
- Pclass: passenger class.
- Name, Sex, Age: self-explanatory
- SibSp: how many siblings & spouses of the passenger aboard the Titanic.
- Parch: how many children & parents of the passenger aboard the Titanic.
- Ticket: ticket id
- Fare: price paid (in pounds)
- Cabin: passenger's cabin number
- Embarked: where the passenger embarked the Titanic

In [8]:
test.head()

PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S

In [9]:
# Explicity make the passenger id the index.
train = train.set_index("PassengerId")
test = test.set_index("PassengerId")

### Data Cleaning.

In [11]:
# Check data type and nulls.
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


In [13]:
def percent_null(data):
    """
    This function outputs columns with missing values with their percentages.
    """
    cols_with_null = list()
    cols = data.columns
    for col in cols:
        null_count = data[col].isnull().value_counts()
        if len(null_count) == 2:
            null_count_percent = np.round((null_count[True]/len(data))*100, 2)
            cols_with_null.append(col)
            print(f"Column {col} has {null_count[True]} missing values which is {null_count_percent}%")
    return cols_with_null
print("Train set has the following information missing:")
cols_with_null_train = percent_null(train)
print(f"\n {'-'*50} \n")
print("Test set has the following information missing:")
cols_with_null_test = percent_null(test)

Train set has the following information missing:
Column Age has 177 missing values which is 19.87%
Column Cabin has 687 missing values which is 77.1%
Column Embarked has 2 missing values which is 0.22%

 -------------------------------------------------- 

Test set has the following information missing:
Column Age has 86 missing values which is 20.57%
Column Fare has 1 missing values which is 0.24%
Column Cabin has 327 missing values which is 78.23%


We will replace missing values based on passenger class and sex.

In [15]:
def fill_missing(data, null_cols):
    """
    Fills numerical columns with missing values using their mean based on grouping.
    Categorical columns are filled using mode.
    """
    for col in null_cols:
        if data[col].dtype == 'object':
            data[col] = data.groupby(["Pclass", "Sex"])[col].transform(lambda x: x.fillna(x.mode()[0]))
        else:
            data[col] = data.groupby(["Pclass", "Sex"])[col].transform(lambda x: x.fillna(x.mean()))
    return data
train = fill_missing(train, cols_with_null_train)
test = fill_missing(test, cols_with_null_test)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     891 non-null    object 
 10  Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       418 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      418 non-null    float64
 8   Cabin     418 non-null    object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


### Let's do some light Exploratory Data Analysis.

In [19]:
# Reduces the number of features.
train["Related"] = train["SibSp"] + train["Parch"]
test["Related"] = test["SibSp"] + test["Parch"]

train.drop(["SibSp", "Parch"], axis=1, inplace=True)
test.drop(["SibSp", "Parch"], axis=1, inplace=True)

In [20]:
survival = train["Survived"].value_counts()
print(f"{survival[1]} people survived whereas {survival[0]} people died representing a {(survival[1]/len(train))*100:.2f}% survival rate")

342 people survived whereas 549 people died representing a 38.38% survival rate


In [21]:
train["Sex"].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [22]:
np.mean(train["Age"])

29.318642716644145

In [23]:
np.mean(train["Fare"])

32.204207968574636

In [24]:
# Calculates aggregates of various columns based on Pclass column.
pclass_group = train.groupby("Pclass").agg(
    pclass_count = pd.NamedAgg(column="Pclass", aggfunc="count"), 
    pclass_age = pd.NamedAgg(column="Age", aggfunc="mean"),
    pclass_fare = pd.NamedAgg(column="Fare", aggfunc="mean"), 
    pclass_survived = pd.NamedAgg(column="Survived", aggfunc="sum"))
pclass_group["pclass_survival_rate"] = np.round((pclass_group["pclass_survived"]/pclass_group["pclass_count"])*100,0)
pclass_group

pclass_count  pclass_age  pclass_fare  pclass_survived  \
Pclass                                                           
1                216   38.378866    84.154687              136   
2                184   29.907295    20.662183               87   
3                491   25.112288    13.675550              119   

        pclass_survival_rate  
Pclass                        
1                       63.0  
2                       47.0  
3                       24.0

In [25]:
# Calculates aggregates of various columns based on Embarked column.
embarked_group = train.groupby("Embarked").agg(
    embarked_count = pd.NamedAgg(column="Embarked", aggfunc="count"), 
    embarked_age = pd.NamedAgg(column="Age", aggfunc="mean"),
    embarked_fare = pd.NamedAgg(column="Fare", aggfunc="mean"), 
    embarked_survived = pd.NamedAgg(column="Survived", aggfunc="sum"))
embarked_group["embarked_survival_rate"] = np.round((embarked_group["embarked_survived"]/embarked_group["embarked_count"])*100,0)
embarked_group

embarked_count  embarked_age  embarked_fare  embarked_survived  \
Embarked                                                                   
C                    168     30.461519      59.954144                 93   
Q                     77     25.690425      13.276030                 30   
S                    646     29.453890      27.243651                219   

          embarked_survival_rate  
Embarked                          
C                           55.0  
Q                           39.0  
S                           34.0

Based on EDA the average fare for the trip was 32 pounds whereas the age was 29 years. As expected, the 1st class passenger paid more than the other class passenger but this margin appears to be very huge. Also, age influenced the ability of a passenger to board better classes with older passengers affording better passenger class. 

People in higher passenger classes seemed to have a better survival rate.

### Preprocessing

In [28]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [29]:
def extract_salutation(name):
    """
    Function to extract salutations people used.
    """
    second_name = name.split(",")[1]
    salutation = second_name.split(".")[0].strip()
    return salutation
train["Salutation"] = train["Name"].apply(extract_salutation)
test["Salutation"] = test["Name"].apply(extract_salutation)

In [ ]:
train.drop("Name", axis=1, inplace=True)
test.drop("Name", axis=1, inplace=True)

In [30]:
categorical_cols = ["Sex", "Ticket", "Cabin", "Embarked", "Salutation"]
numeric_cols = ["Pclass", "Age", "Related", "Fare"]

In [31]:
# Checks if the newly added column adds valueable information in relation to survival.
salutation_group = train.groupby("Salutation").agg(
    salutation_count = pd.NamedAgg(column="Salutation", aggfunc="count"), 
    salutation_age = pd.NamedAgg(column="Age", aggfunc="mean"),
    salutation_fare = pd.NamedAgg(column="Fare", aggfunc="mean"), 
    salutation_survived = pd.NamedAgg(column="Survived", aggfunc="sum"))
salutation_group["salutation_survival_rate"] = np.round((salutation_group["salutation_survived"] /
                                                         salutation_group["salutation_count"])*100,0)
salutation_group

salutation_count  salutation_age  salutation_fare  \
Salutation                                                        
Capt                         1       70.000000        71.000000   
Col                          2       58.000000        31.025000   
Don                          1       40.000000        27.720800   
Dr                           7       41.897341        49.168457   
Jonkheer                     1       38.000000         0.000000   
Lady                         1       48.000000        39.600000   
Major                        2       48.500000        28.525000   
Master                      40        6.767509        34.703125   
Miss                       182       21.916526        43.797873   
Mlle                         2       24.000000        59.402100   
Mme                          1       24.000000        69.300000   
Mr                         517       31.664366        24.441560   
Mrs                        125       34.797153        45.138533   
Ms                           1       28.000000        13.000000   
Rev                          6       43.166667        18.312500   
Sir                          1       49.000000        56.929200   
the Countess                 1       33.000000        86.500000   

              salutation_survived  salutation_survival_rate  
Salutation                                                   
Capt                            0                       0.0  
Col                             1                      50.0  
Don                             0                       0.0  
Dr                              3                      43.0  
Jonkheer                        0                       0.0  
Lady                            1                     100.0  
Major                           1                      50.0  
Master                         23                      57.0  
Miss                          127                      70.0  
Mlle                            2                     100.0  
Mme                             1                     100.0  
Mr                             81                      16.0  
Mrs                            99                      79.0  
Ms                              1                     100.0  
Rev                             0                       0.0  
Sir                             1                     100.0  
the Countess                    1                     100.0

In [32]:
# Transforms age into categorical column.
train["AgeBucket"] = train["Age"] // 15 * 15
test["AgeBucket"] = test["Age"] // 15 * 15

In [33]:
# Checks if the newly added column adds valueable information in relation to survival. 
group = train[["AgeBucket", "Survived"]].groupby("AgeBucket").mean().sort_values(by="Survived")
count = train["AgeBucket"].value_counts()
agebucket_survival_count_merge = group.merge(count, how="left", on="AgeBucket")

In [34]:
agebucket_survival_count_merge

Survived  count
AgeBucket                 
60.0       0.240000     25
15.0       0.331081    444
45.0       0.404494     89
30.0       0.421260    254
0.0        0.576923     78
75.0       1.000000      1

In [44]:
train

Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   
...               ...     ...   
887                 0       2   
888                 1       1   
889                 0       3   
890                 1       1   
891                 0       3   

                                                          Name     Sex    Age  \
PassengerId                                                                     
1                                      Braund, Mr. Owen Harris    male  22.00   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.00   
3                                       Heikkinen, Miss. Laina  female  26.00   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.00   
5                                     Allen, Mr. William Henry    male  35.00   
...                                                        ...     ...    ...   
887                                      Montvila, Rev. Juozas    male  27.00   
888                               Graham, Miss. Margaret Edith  female  19.00   
889                   Johnston, Miss. Catherine Helen "Carrie"  female  21.75   
890                                      Behr, Mr. Karl Howell    male  26.00   
891                                        Dooley, Mr. Patrick    male  32.00   

                       Ticket     Fare  Cabin Embarked  Related Salutation  \
PassengerId                                                                  
1                   A/5 21171   7.2500  F G73        S        1         Mr   
2                    PC 17599  71.2833    C85        C        1        Mrs   
3            STON/O2. 3101282   7.9250     G6        S        0       Miss   
4                      113803  53.1000   C123        S        1        Mrs   
5                      373450   8.0500  F G73        S        0         Mr   
...                       ...      ...    ...      ...      ...        ...   
887                    211536  13.0000     F2        S        0        Rev   
888                    112053  30.0000    B42        S        0       Miss   
889                W./C. 6607  23.4500     G6        S        3       Miss   
890                    111369  30.0000   C148        C        0         Mr   
891                    370376   7.7500  F G73        Q        0         Mr   

             AgeBucket  
PassengerId             
1                 15.0  
2                 30.0  
3                 15.0  
4                 30.0  
5                 30.0  
...                ...  
887               15.0  
888               15.0  
889               15.0  
890               15.0  
891               30.0  

[891 rows x 12 columns]

In [35]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
one_hot_encoder.fit(train[categorical_cols])
one_hot_train = one_hot_encoder.transform(train[categorical_cols])
one_hot_test = one_hot_encoder.transform(test[categorical_cols])
one_hot_train = one_hot_train.toarray()
one_hot_test = one_hot_test.toarray()

ValueError: Found unknown categories ['Johansson, Mr. Nils', 'McCarthy, Miss. Catherine Katie""', 'Samaan, Mr. Elias', 'Moubarek, Mrs. George (Omine Amenia" Alexander)"', 'Katavelas, Mr. Vassilios (Catavelas Vassilios")"', 'Carrau, Mr. Jose Pedro', 'Omont, Mr. Alfred Fernand', 'Risien, Mrs. Samuel (Emma)', 'Canavan, Mr. Patrick', 'Geiger, Miss. Amalie', 'Cassebeer, Mrs. Henry Arthur Jr (Eleanor Genevieve Fosdick)', 'Williams, Mr. Richard Norris II', 'Frolicher-Stehli, Mrs. Maxmillian (Margaretha Emerentia Stehli)', 'Khalil, Mr. Betros', 'Drew, Master. Marshall Brines', 'Kink-Heilmann, Mr. Anton', 'Kink, Miss. Maria', 'Kiernan, Mr. John', 'Ryerson, Master. John Borie', 'Ware, Mr. William Jeffery', 'Ford, Mr. Edward Watson', 'Aks, Master. Philip Frank', 'Brown, Miss. Edith Eileen', 'Rowe, Mr. Alfred G', 'Hyman, Mr. Abraham', 'Mulvihill, Miss. Bertha E', 'Riihivouri, Miss. Susanna Juhantytar Sanni""', 'Hiltunen, Miss. Marta', 'Carver, Mr. Alfred John', 'Jefferys, Mr. Ernest Wilfred', 'Hoc

In [ ]:
y_train = train["Survived"].to_numpy()

In [ ]:
X_train = np.c_[train[numeric_cols], one_hot_train]
X_test = np.c_[test[numeric_cols], one_hot_test]

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

### Build some ML models.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

Let's use grid search to search for hyperparameters.

In [ ]:
svm_clf = LinearSVC()

In [ ]:
params = [{"C": [1, 2, 3, 5], "dual": ["auto", False]}]

In [ ]:
grid_search = GridSearchCV(svm_clf, params, scoring="accuracy", verbose=3)
grid_search.fit(X_train_scaled, y_train)

In [ ]:
print(grid_search.best_params_)

In [ ]:
cvres = grid_search.cv_results_
for mean_test_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(f"mean_test_score: {mean_test_score} and params: {params}")

In [ ]:
print(cvres["mean_test_score"].max())

In [ ]:
predictions = grid_search.predict(X_test_scaled)

In [ ]:
df = pd.DataFrame()
df["PassengerId"] = np.arange(892, 1310)
df["Survived"] = predictions
df.to_csv("titanic_svm.csv", index=False)